In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt             #visualisation
import seaborn as sns   #visualisation
%matplotlib inline
sns.set(color_codes=True)

In [2]:
#read the data
#laptop
#df_DropSeq_MCF7 = pd.read_csv("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\DropSeq\\MCF7_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ", engine='python', index_col=0)
#desktop
#C:\Users\Cameron\Desktop\AILab24\DropSeq
df_DropSeq_MCF7 = pd.read_csv("C:\\Users\\Cameron\\Desktop\\AILab24\\DropSeq\\MCF7_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ", engine='python', index_col=0)

In [3]:
#add a new row to the data frame
#add the row to the bottom
#if at the column index contains _Normoxia then add 1 to the row, _Hypoxia then add -1
#this is just binary classification

In [4]:
#function to add new 'Oxia' row to the data frame
#1 is Normoxia, 0 is Hypoxia
def Oxia(df):
    new_row = {col: None for col in df.columns}
    for col in df.columns:
        if '_Normoxia' in col:
            new_row[col] = 1
        elif '_Hypoxia' in col:
            new_row[col] = 0
    df = pd.concat([df, pd.DataFrame(new_row, index=['oxia'])])
    return df

In [5]:
df_DropSeq_MCF7 = Oxia(df_DropSeq_MCF7)
df_DropSeq_MCF7

,"""AAAAACCTATCG_Normoxia""","""AAAACAACCCTA_Normoxia""","""AAAACACTCTCA_Normoxia""","""AAAACCAGGCAC_Normoxia""","""AAAACCTAGCTC_Normoxia""","""AAAACCTCCGGG_Normoxia""","""AAAACTCGTTGC_Normoxia""","""AAAAGAGCTCTC_Normoxia""","""AAAAGCTAGGCG_Normoxia""","""AAAATCGCATTT_Normoxia""",...,"""TTTTACAGGATC_Hypoxia""","""TTTTACCACGTA_Hypoxia""","""TTTTATGCTACG_Hypoxia""","""TTTTCCAGACGC_Hypoxia""","""TTTTCGCGCTCG_Hypoxia""","""TTTTCGCGTAGA_Hypoxia""","""TTTTCGTCCGCT_Hypoxia""","""TTTTCTCCGGCT_Hypoxia""","""TTTTGTTCAAAG_Hypoxia""","""TTTTTTGTATGT_Hypoxia"""
"""MALAT1""",1,3,3,6,4,5,1,13,3,3,...,0,2,1,0,1,0,1,0,0,4
"""MT-RNR2""",0,0,0,2,0,0,2,1,7,0,...,0,0,0,0,0,0,0,0,0,0
"""NEAT1""",0,0,0,0,0,2,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0
"""H1-5""",0,0,0,0,0,2,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
"""TFF1""",4,1,1,1,0,0,0,2,0,1,...,2,3,8,0,0,3,4,2,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"""RPS19BP1""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""AUNIP""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""TNK2""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""SUDS3""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
#now to the data
#transpose the data
df_DropSeq_MCF7 = df_DropSeq_MCF7.T
df_DropSeq_MCF7

,"""MALAT1""","""MT-RNR2""","""NEAT1""","""H1-5""","""TFF1""","""MT-RNR1""","""H4C3""","""GDF15""","""KRT81""","""MT-CO3""",...,"""SKIDA1""","""MICALL1""","""RARG""","""MYO1F""","""BRWD1-AS2""","""RPS19BP1""","""AUNIP""","""TNK2""","""SUDS3""",oxia
"""AAAAACCTATCG_Normoxia""",1,0,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"""AAAACAACCCTA_Normoxia""",3,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"""AAAACACTCTCA_Normoxia""",3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"""AAAACCAGGCAC_Normoxia""",6,2,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"""AAAACCTAGCTC_Normoxia""",4,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"""TTTTCGCGTAGA_Hypoxia""",0,0,0,0,3,0,7,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""TTTTCGTCCGCT_Hypoxia""",1,0,0,0,4,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
"""TTTTCTCCGGCT_Hypoxia""",0,0,0,1,2,0,4,0,0,0,...,0,0,0,0,0,0,0,0,1,0
"""TTTTGTTCAAAG_Hypoxia""",0,0,0,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#count the number of hypoxic and normoxic samples
df_DropSeq_MCF7['oxia'].value_counts()

oxia
1    12705
0     8921
Name: count, dtype: int64

In [8]:
#now we need to split the data into training and testing data
#remove the oxia row and store it in a variable
y_DropSeq_MCF7 = df_DropSeq_MCF7['oxia']
X_DropSeq_MCF7 = df_DropSeq_MCF7.drop('oxia', axis=1)

In [9]:
X_DropSeq_MCF7 #this is all the data, needs to be split for testing and training

,"""MALAT1""","""MT-RNR2""","""NEAT1""","""H1-5""","""TFF1""","""MT-RNR1""","""H4C3""","""GDF15""","""KRT81""","""MT-CO3""",...,"""MROH1""","""SKIDA1""","""MICALL1""","""RARG""","""MYO1F""","""BRWD1-AS2""","""RPS19BP1""","""AUNIP""","""TNK2""","""SUDS3"""
"""AAAAACCTATCG_Normoxia""",1,0,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""AAAACAACCCTA_Normoxia""",3,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""AAAACACTCTCA_Normoxia""",3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""AAAACCAGGCAC_Normoxia""",6,2,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""AAAACCTAGCTC_Normoxia""",4,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"""TTTTCGCGTAGA_Hypoxia""",0,0,0,0,3,0,7,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""TTTTCGTCCGCT_Hypoxia""",1,0,0,0,4,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
"""TTTTCTCCGGCT_Hypoxia""",0,0,0,1,2,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"""TTTTGTTCAAAG_Hypoxia""",0,0,0,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
y_DropSeq_MCF7 #this is the target data, were trying to predict this

"AAAAACCTATCG_Normoxia"    1
"AAAACAACCCTA_Normoxia"    1
"AAAACACTCTCA_Normoxia"    1
"AAAACCAGGCAC_Normoxia"    1
"AAAACCTAGCTC_Normoxia"    1
                          ..
"TTTTCGCGTAGA_Hypoxia"     0
"TTTTCGTCCGCT_Hypoxia"     0
"TTTTCTCCGGCT_Hypoxia"     0
"TTTTGTTCAAAG_Hypoxia"     0
"TTTTTTGTATGT_Hypoxia"     0
Name: oxia, Length: 21626, dtype: int64

In [11]:
import sklearn
from sklearn.model_selection import train_test_split
#split the data into training and testing data
X_train_DropSeq_MCF7, X_test_DropSeq_MCF7, y_train_DropSeq_MCF7, y_test_DropSeq_MCF7 = train_test_split(X_DropSeq_MCF7, y_DropSeq_MCF7, test_size=0.2, random_state=42)
#lets see how they are split

In [12]:
#and their shapes:
print("X_train shape: ", X_train_DropSeq_MCF7.shape, "X_test shape: ", X_test_DropSeq_MCF7.shape, "y_train shape: ", y_train_DropSeq_MCF7.shape, "y_test shape: ", y_test_DropSeq_MCF7.shape)

X_train shape:  (17300, 3000) X_test shape:  (4326, 3000) y_train shape:  (17300,) y_test shape:  (4326,)


In [13]:
#now to train the model
#lets start with simple linear regression and see how it goes
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Step 1: Initialize the Linear Regression model
model_linreg_DropSeq_MCF7 = LinearRegression()

# Step 2: Fit the model on the training data
model_linreg_DropSeq_MCF7.fit(X_train_DropSeq_MCF7, y_train_DropSeq_MCF7)

# Step 3: Predict the target variable for the testing data
y_pred_lin_DropSeq_MCF7 = model_linreg_DropSeq_MCF7.predict(X_test_DropSeq_MCF7)

# Step 4: Evaluate the model's performance
mse = mean_squared_error(y_test_DropSeq_MCF7, y_pred_lin_DropSeq_MCF7)
r2 = r2_score(y_test_DropSeq_MCF7, y_pred_lin_DropSeq_MCF7)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 0.06182092653045139
R^2 Score: 0.7438691949867688


In [14]:
#because we already have binary output data it might be worth to try a logistic regression model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
model_logistic_DropSeq_MCF7 = LogisticRegression(solver='liblinear')
params_log_DropSeq_MCF7 = {'penalty': ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
model_logistic_gs_DropSeq_MCF7 = GridSearchCV(model_logistic_DropSeq_MCF7, params_log_DropSeq_MCF7, cv=5, scoring=['neg_log_loss'], refit='neg_log_loss')
model_logistic_gs_DropSeq_MCF7.fit(X_train_DropSeq_MCF7, y_train_DropSeq_MCF7)

GridSearchCV(cv=5, estimator=LogisticRegression(solver='liblinear'),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'penalty': ['l1', 'l2']},
             refit='neg_log_loss', scoring=['neg_log_loss'])

In [15]:
best_log_model_DropSeq_MCF7 = model_logistic_gs_DropSeq_MCF7.best_estimator_
print(best_log_model_DropSeq_MCF7)
print(f"The best parameters for the logistic regression:{model_logistic_gs_DropSeq_MCF7.best_params_}")

LogisticRegression(C=0.1, solver='liblinear')
The best parameters for the logistic regression:{'C': 0.1, 'penalty': 'l2'}


In [16]:
#How accurate were the predictions?
y_pred_log = best_log_model_DropSeq_MCF7.predict(X_test_DropSeq_MCF7)
accuracy_logistic = accuracy_score(y_test_DropSeq_MCF7, y_pred_log)
print(f"Accuracy: {accuracy_logistic}")

Accuracy: 0.9787332408691632


In [21]:
#k nearest neighbours
from sklearn.neighbors import KNeighborsClassifier
knn_DropSeq_MCF7 = KNeighborsClassifier()
params_knn_DropSeq_MCF7 = {'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance']}
knn_gs_DropSeq_MCF7 = GridSearchCV(knn_DropSeq_MCF7, params_knn_DropSeq_MCF7, cv=5, scoring=['neg_log_loss'], refit='neg_log_loss')
knn_gs_DropSeq_MCF7.fit(X_train_DropSeq_MCF7, y_train_DropSeq_MCF7)
best_knn_DropSeq_MCF7 = knn_gs_DropSeq_MCF7.best_estimator_

ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.

In [ ]:
best_knn_DropSeq_MCF7.fit(X_train_DropSeq_MCF7, y_train_DropSeq_MCF7)
y_knn_pred_DropSeq_MCF7 = best_knn_DropSeq_MCF7.predict(X_test_DropSeq_MCF7)

In [ ]:
print(f"Accuracy: {accuracy_score(y_test_DropSeq_MCF7, y_knn_pred_DropSeq_MCF7)}")

In [ ]:
#random forest
from  sklearn.ensemble import RandomForestClassifier
Forest_DropSeq_MCF7 = RandomForestClassifier(random_state=42)
params_forest_DropSeq_MCF7 = {'n_estimators': [10, 50, 100, 200], 'max_depth': [10, 50, 100, 200]}
forest_gs_DropSeq_MCF7 = GridSearchCV(Forest_DropSeq_MCF7, params_forest_DropSeq_MCF7, cv=5, scoring=['neg_log_loss'], refit='neg_log_loss')
forest_gs_DropSeq_MCF7.fit(X_train_DropSeq_MCF7, y_train_DropSeq_MCF7)
best_forest_DropSeq_MCF7 = forest_gs_DropSeq_MCF7.best_estimator_

In [ ]:
best_forest_DropSeq_MCF7.fit(X_train_DropSeq_MCF7, y_train_DropSeq_MCF7)
y_forest_pred_DropSeq_MCF7 = best_forest_DropSeq_MCF7.predict(X_test_DropSeq_MCF7)

In [ ]:
print(f"Accuracy: {accuracy_score(y_test_DropSeq_MCF7, y_forest_pred_DropSeq_MCF7)}")

In [ ]:
#now to try a different model
#how about perceptron multilayer perceptron
from sklearn.neural_network import MLPClassifier

# Step 1: Initialize the MLP Classifier model
NN_DropSeq_MCF7 = MLPClassifier(random_state=42, batch_size='auto', max_iter=1000000, solver='sgd')
# Step 2: Fit the model on the training data
params_grid = {'hidden_layer_sizes': [(50,), (100,), (50,50), (100,100)], 'learning_rate_init': [0.1, 0.01, 0.001]}
NN_gs_DropSeq_MCF7 = GridSearchCV(NN_DropSeq_MCF7, params_grid, cv=3, verbose=2)
NN_gs_DropSeq_MCF7.fit(X_train_DropSeq_MCF7, y_train_DropSeq_MCF7)
NN_best_DropSeq_MCF7 = NN_gs_DropSeq_MCF7.best_estimator_
NN_best_DropSeq_MCF7.fit(X_train_DropSeq_MCF7, y_train_DropSeq_MCF7)

In [ ]:
# Step 3: Predict the target variable for the testing data
y_NN_pred_DropSeq_MCF7 = NN_best_DropSeq_MCF7.predict(X_test_DropSeq_MCF7)
#step 4: Evaluate the model's performance
from sklearn.metrics import accuracy_score
print(f"Accuracy: {accuracy_score(y_test_DropSeq_MCF7, y_NN_pred_DropSeq_MCF7)}")

In [ ]:
#ensembling
from sklearn.ensemble import VotingClassifier
#create a dictionary of our models
estimators_DropSeq_MCF7 = [('logistic', best_log_model_DropSeq_MCF7), ('forest', best_forest_DropSeq_MCF7), ('NN', NN_best_DropSeq_MCF7)]
#create our voting classifier, inputting our models
ensemble_DropSeq_MCF7 = VotingClassifier(estimators_DropSeq_MCF7, voting='soft')
#fit model to training data
ensemble_DropSeq_MCF7.fit(X_train_DropSeq_MCF7, y_train_DropSeq_MCF7)

In [ ]:
#predict the target variable for the testing data
y_ensemble_pred_DropSeq_MCF7 = ensemble_DropSeq_MCF7.predict(X_test_DropSeq_MCF7)
#evaluate the model's performance
print(f"Accuracy: {accuracy_score(y_test_DropSeq_MCF7, y_ensemble_pred_DropSeq_MCF7)}")

In [ ]:
#what if we try with the hcc data
#laptop
#df = pd.read_csv("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\DropSeq\\HCC1806_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ", engine='python', index_col=0)
#desktop
#C:\Users\Cameron\Desktop\AILab24\DropSeq
df_DropSeq_HCC = pd.read_csv("C:\\Users\\Cameron\\Desktop\\AILab24\\DropSeq\\HCC1806_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ", engine='python', index_col=0)
df_DropSeq_HCC

In [ ]:
#same process as before
df_DropSeq_HCC = Oxia(df_DropSeq_HCC)
df_DropSeq_HCC

In [ ]:
df_DropSeq_HCC = df_DropSeq_HCC.T
df_DropSeq_HCC

In [ ]:
df_DropSeq_HCC['oxia'].value_counts()

In [ ]:
y_DropSeq_HCC = df_DropSeq_HCC['oxia']
X_DropSeq_HCC = df_DropSeq_HCC.drop('oxia', axis=1)

In [ ]:
X_train_DropSeq_HCC, X_test_DropSeq_HCC, y_train_DropSeq_HCC, y_test_DropSeq_HCC = train_test_split(X_DropSeq_HCC, y_DropSeq_HCC, test_size=0.2, random_state=42)

In [ ]:
#try the same methods as before, first a linear regression, then try a logistic regression
model_linreg_DropSeq_HCC = LinearRegression()
model_linreg_DropSeq_HCC.fit(X_train_DropSeq_HCC, y_train_DropSeq_HCC)
y_pred_lin_DropSeq_HCC = model_linreg_DropSeq_HCC.predict(X_test_DropSeq_HCC)
mse = mean_squared_error(y_test_DropSeq_HCC, y_pred_lin_DropSeq_HCC)
r2 = r2_score(y_test_DropSeq_HCC, y_pred_lin_DropSeq_HCC)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

In [ ]:
model_logistic_DropSeq_HCC = LogisticRegression(solver='liblinear')
params_log_DropSeq_HCC = {'penalty': ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
model_logistic_gs_DropSeq_HCC = GridSearchCV(model_logistic_DropSeq_HCC, params_log_DropSeq_HCC, cv=5, scoring=['neg_log_loss'], refit='neg_log_loss')
model_logistic_gs_DropSeq_HCC.fit(X_train_DropSeq_HCC, y_train_DropSeq_HCC)
best_log_model_DropSeq_HCC = model_logistic_gs_DropSeq_HCC.best_estimator_
print(best_log_model_DropSeq_HCC)
print(f"The best parameters for the logistic regression:{model_logistic_gs_DropSeq_HCC.best_params_}")

In [ ]:
y_pred_log = best_log_model_DropSeq_HCC.predict(X_test_DropSeq_HCC)
accuracy_logistic = accuracy_score(y_test_DropSeq_HCC, y_pred_log)
print(f"Accuracy: {accuracy_logistic}")

In [ ]:
#k nearest neighbors
knn_DropSeq_HCC = KNeighborsClassifier()
params_knn_DropSeq_HCC = {'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance']}
knn_gs_DropSeq_HCC = GridSearchCV(knn_DropSeq_HCC, params_knn_DropSeq_HCC, cv=5, scoring=['neg_log_loss'], refit='neg_log_loss')
knn_gs_DropSeq_HCC.fit(X_train_DropSeq_HCC, y_train_DropSeq_HCC)
best_knn_DropSeq_HCC = knn_gs_DropSeq_HCC.best_estimator_

In [ ]:
best_knn_DropSeq_HCC.fit(X_train_DropSeq_HCC, y_train_DropSeq_HCC)
y_knn_pred_DropSeq_HCC = best_knn_DropSeq_HCC.predict(X_test_DropSeq_HCC)

In [ ]:
print(f"Accuracy: {accuracy_score(y_test_DropSeq_HCC, y_knn_pred_DropSeq_HCC)}")

In [ ]:
#random forest
Forest_DropSeq_HCC = RandomForestClassifier(random_state=42)
params_forest_DropSeq_HCC = {'n_estimators': [10, 50, 100, 200], 'max_depth': [10, 50, 100, 200]}
forest_gs_DropSeq_HCC = GridSearchCV(Forest_DropSeq_HCC, params_forest_DropSeq_HCC, cv=5, scoring=['neg_log_loss'], refit='neg_log_loss')
forest_gs_DropSeq_HCC.fit(X_train_DropSeq_HCC, y_train_DropSeq_HCC)
best_forest_DropSeq_HCC = forest_gs_DropSeq_HCC.best_estimator_

In [ ]:
best_forest_DropSeq_HCC.fit(X_train_DropSeq_HCC, y_train_DropSeq_HCC)
y_forest_pred_DropSeq_HCC = best_forest_DropSeq_HCC.predict(X_test_DropSeq_HCC)

In [ ]:
print(f"Accuracy: {accuracy_score(y_test_DropSeq_HCC, y_forest_pred_DropSeq_HCC)}")

In [ ]:
#time for a neural network
NN_hcc = MLPClassifier(random_state=42, batch_size='auto', max_iter=1000000, solver='sgd')
params_grid = {'hidden_layer_sizes': [(50,), (100,), (50,50), (100,100)], 'learning_rate_init': [0.1, 0.01, 0.001]}
NN_gs_DropSeq_HCC = GridSearchCV(NN_hcc, params_grid, cv=3, verbose=2)
NN_gs_DropSeq_HCC.fit(X_train_DropSeq_HCC, y_train_DropSeq_HCC)
NN_best_DropSeq_HCC = NN_gs_DropSeq_HCC.best_estimator_

In [ ]:
NN_best_DropSeq_HCC.fit(X_train_DropSeq_HCC, y_train_DropSeq_HCC)
y_NN_pred_DropSeq_HCC = NN_best_DropSeq_HCC.predict(X_test_DropSeq_HCC)

In [ ]:
print(f"Accuracy: {accuracy_score(y_test_DropSeq_HCC, y_NN_pred_DropSeq_HCC)}")

In [ ]:
#ensembling
estimators_DropSeq_HCC = [('logistic', best_log_model_DropSeq_HCC), ('forest', best_forest_DropSeq_HCC), ('NN', NN_best_DropSeq_HCC)]
ensemble_DropSeq_HCC = VotingClassifier(estimators_DropSeq_HCC, voting='soft')
ensemble_DropSeq_HCC.fit(X_train_DropSeq_HCC, y_train_DropSeq_HCC)

In [ ]:
y_ensemble_pred_DropSeq_HCC = ensemble_DropSeq_HCC.predict(X_test_DropSeq_HCC)
print(f"Accuracy: {accuracy_score(y_test_DropSeq_HCC, y_ensemble_pred_DropSeq_HCC)}")